# Before running the program ensure that:
#### 1. Download the latest Analytical Report
#### 2. Adjust dates in Analytical Report
#### 3. Download Actuals Report
#### 4. Run Actuals for the desired book
#### 5. Define Dates

In [1]:
import numpy as np
import pandas as pd
import datetime

import warnings
warnings.filterwarnings("ignore")

# Define Dates

In [2]:
# define radar date

radar_date = '2017-09-12'
end_date = '2017-10-31'
#add_days = pd.to_datetime(end_date) - pd.to_datetime(radar_date)
add_days = 49
add_days

49

# Import Analytical Report

In [3]:
df = pd.read_excel('0FLATANALYTICALexport.XLSX')

# Counterparties and dates

In [4]:
# correction of counterparty (+)
df['Counterparty (+)'] = np.where((df['Mirror portfolio name (+)'].isnull() == True ),
                                  df['Counterparty (+)'],
                                  np.where(df['Mirror Line of Business (+)'] == df['Line of Business'],
                                          df['Mirror Line of Business (+)'].str[:3] + ' ' + df['Mirror portfolio name (+)'],
                                          df['Mirror Line of Business (+)'] + ' ' + df['Mirror portfolio name (+)']))

In [5]:
# correction of counterparty (-)
df['Counterparty (-)'] = np.where((df['Mirror portfolio name (-)'].isnull() == True ),
                                  df['Counterparty (-)'],
                                 np.where(df['Mirror Line of Business (-)'] == df['Line of Business'],
                                         df['Mirror Line of Business (-)'].str[:3] + ' '+ df['Mirror portfolio name (-)'],
                                          df['Mirror Line of Business (-)'] + ' ' + df['Mirror portfolio name (-)']))

In [6]:
#correction of Due date (-)
df['Due date (-)'] = df['Due date (-)'].astype('datetime64[ns]')
df['Due date (-)'][df['Due date (-)'] == '1816-03-29 05:56:08.066277376' ] = 0
df['test'] = np.where(df['Due date (-)'] == '1970-01-01', 0, 1)

# correction of Title transfer date (-)
df['Title transfer date (-)'] = df['Title transfer date (-)'].astype(object).where(df['Title transfer date (-)'].notnull(), 0)
df['Title transfer date (-)'] = np.where(df['Title transfer date (-)'] == 0, pd.to_datetime('today') + np.timedelta64(30,'D') , df['Title transfer date (-)'])
df['Title transfer date (-)'] = df['Title transfer date (-)'].astype('datetime64[ns]')

#continuation of correction of Due date (-)
df['Due date (-) corrected'] = np.where(df['test'] == 1, df['Due date (-)'], df['Due date (+)'] - df['Title transfer date (+)'] + df['Title transfer date (-)'])

df['Due date (-)'] = df['Due date (-) corrected']

# converting the type of Due date (-)
df['Due date (-)'] = df['Due date (-)'].astype('datetime64[ns]')
df['Due date (-)'] = df['Due date (-)'].dt.date

df['Consumption Date'] = df['Due date (+)']
df['Due date (+)'] = df['Due date (+)'].dt.date
df['Title transfer date (-)'] = df['Title transfer date (-)'].dt.date
df['Title transfer date (+)'] = df['Title transfer date (+)'].dt.date


#filling NaN
df['Incoterms 1 (-)'].fillna(df['Incoterms 1 (+)'], inplace=True)
df['Incoterms 2 country (-)'].fillna(df['Incoterms 2 country (+).1'], inplace=True)
df['Incoterms 2 country (-).1'].fillna(df['Incoterms 2 country (+)'], inplace=True)


# Days operations

In [7]:
# calculating the columns

df['Consumption Type'] = 'Filled by Macros'
df['Total WC Consumption period'] = df['Due date (-)'] - df['Due date (+)']
df['TTD difference'] = df['Title transfer date (-)'] - df['Title transfer date (+)']
df['Sale Payment Terms'] = df['Due date (-)'] - df['Title transfer date (-)']
df['Buy Payment Terms'] = df['Due date (+)'] - df['Title transfer date (+)']
df['PDD difference'] = df['Sale Payment Terms'] - df['Buy Payment Terms']
df['Consumption reason'] = 'OTHER'
df['WC Amount'] = df['WC consumption amount'] / 1000000
df['Current amount in USD (+)'] = df['Quantity BBL (ratio) (+)'] * df['Current price, USD/BBL (+)']
df['Current amount in USD (-)'] = df['Quantity BBL (ratio) (-)'] * df['Current price, USD/BBL (-)']

# company name
df['Company Name'] = df['Line of Business'].str[:3]

# portfolio name
df['Portfolio name change'] = df['Company Name'] + ' ' + df['Portfolio name']
df['Portfolio name'] = df['Portfolio name change']


# converting columns to the right format
#converting into days
df['Total WC Consumption period'] = df['Total WC Consumption period'].dt.days
df['TTD difference'] = df['TTD difference'].dt.days
df['Sale Payment Terms'] = df['Sale Payment Terms'].dt.days
df['Buy Payment Terms'] = df['Buy Payment Terms'].dt.days
df['PDD difference'] = df['PDD difference'].dt.days

# WC Amount

In [8]:
# correcting the WC Amount
df_strategy = pd.DataFrame()
df_strategy = df[['Strategy', 'WC Amount','Strategy start date']].copy()

#import the strategies
df_strategy_exclude = pd.read_excel('Strategies.xlsx')

#conditions and sum
df_strategy = df_strategy.merge(df_strategy_exclude, on = 'Strategy', how = 'left')
df_strategy['New'].fillna(0, inplace=True)
df_strategy['S2'] = np.where(df_strategy['Strategy'] > 517000, 1,0)
df_strategy['ssd'] = np.where(df_strategy['Strategy start date'] > '2015-12-01', 1,0)
df_strategy['sum_s'] = df_strategy['New'] + df_strategy['ssd'] + df_strategy['S2']
df_strategy['WC Amount corrected'] = np.where(df_strategy['sum_s'] > 0,df_strategy['WC Amount'], 0)

# replacing WC amount
df['WC Amount'] = df_strategy['WC Amount corrected']

# rounding WC amount and filter
df['WC Amount'] = df['WC Amount'].round(4)
df = df[(df['WC Amount'] <= -0.50) | (df['WC Amount'] >= 0.50) ]

# reset index
df = df.reset_index(drop=True)

# WC Days: Total, TTD and PDD

In [9]:
def f(row):
    if (row['Total WC Consumption period'] >= row['TTD difference'] and row['Total WC Consumption period'] > 0 and row['TTD difference'] > 0) \
    or (row['Total WC Consumption period'] <= row['TTD difference'] and row['Total WC Consumption period'] < 0 and row['TTD difference'] < 0):
        return row['TTD difference']
    elif (row['Total WC Consumption period'] > 0 and row['TTD difference'] > 0) \
    or (row['Total WC Consumption period'] < 0 and row['TTD difference'] < 0):
        return row['Total WC Consumption period']
    else:
        return 0

In [10]:
df['TTD influence on WC consumption'] = df.apply(f, axis=1)
df['PDD influence on WC consumption'] = df['Total WC Consumption period'] - df['TTD influence on WC consumption']

# Select columns

In [11]:
# rearrange columns
cols = df.columns.tolist()

In [12]:
cols = ['Consumption Date',
       'Consumption Type',
       'Total WC Consumption period',
        'TTD influence on WC consumption',
        'PDD influence on WC consumption',
        'TTD difference',
        'PDD difference',
        'Sale Payment Terms',
        'Buy Payment Terms',
        'Consumption reason',
        'WC Amount',
        'Buy/Sell (-)',
        'Buy/Sell (+)',
        'Due date (-)',
        'Due date (+)',
        'Title transfer date (-)',
        'Title transfer date (+)',
        'Strategy',
        'Strategy name',
        'Company Name',
        'Global Book name',
        'Line of Business',
        'Book Name',
        'Portfolio name',
        'Quantity',
        'Current amount in USD (+)',
        'Quantity BBL (ratio) (+)',
        'Current price, USD/BBL (+)',
        'Counterparty (+)',
        'Title transfer (+)',
        'Incoterms 1 (+)',
        'Incoterms 2 country (+)',
        'Incoterms 2 country (+).1',
        'Object type (+)',
        'Delivery ID (+)',
        'Internal type (+)',
        'Mirror Delivery ID (+)',
        'Current amount in USD (-)',
        'Quantity BBL (ratio) (-)',
        'Current price, USD/BBL (-)',
        'Counterparty (-)',
        'Title transfer (-)',
        'Incoterms 1 (-)',
        'Incoterms 2 country (-)',
        'Incoterms 2 country (-).1',
        'Object type (-)',
        'Delivery ID (-)',
        'Internal type (-)',
        'Mirror Delivery ID (-)',
        'Strategy start date',
        'Strategy end date',
        'Voyage ID (+)',
        'Transport name (+)',
        'Grade (+)',
        'Voyage ID (-)',
        'Transport name (-)',
        'Grade (-)']

In [13]:
# ordering the columns
df = df[cols]
df.shape

(5951, 57)

# Filter data

In [14]:
# selecting rows for which dates are above -365 or below 365
df = df[(df['Total WC Consumption period'] > -365) & (df['Total WC Consumption period'] < 365) ]

# reset index
df = df.reset_index(drop=True)
df.shape

(5943, 57)

# Stock in Transit and Payment Terms

In [15]:
# stock in transit
stock_in_transit = df.loc[df.index.repeat(abs(df['TTD influence on WC consumption']))]
stock_in_transit['Consumption Date'] = stock_in_transit['Consumption Date'] + pd.to_timedelta(abs(stock_in_transit['TTD influence on WC consumption'])-1, 'D')

stock_in_transit.loc[:,'Consumption Date'] -= pd.to_timedelta(stock_in_transit.groupby(level=0).cumcount(),'D')
stock_in_transit = stock_in_transit.reset_index(drop=True)
stock_in_transit['Consumption Type'] = 'Stock in Transit'
stock_in_transit = stock_in_transit[(stock_in_transit['Consumption Date'] > '2016-12-31')]

# correct the negatives
stock_in_transit['Consumption Date'] = np.where(stock_in_transit['TTD influence on WC consumption'] < 0,
                                                stock_in_transit['Consumption Date'] +
                                                pd.to_timedelta(stock_in_transit['TTD influence on WC consumption'], 'D'),
                                                stock_in_transit['Consumption Date'])

#filter date
stock_in_transit = stock_in_transit[(stock_in_transit['Consumption Date'] > '2016-12-31')]
stock_in_transit = stock_in_transit.reset_index(drop=True)

stock_in_transit.shape

(30734, 57)

In [16]:
# payment terms
payment_terms = df.loc[df.index.repeat(abs(df['PDD influence on WC consumption']))]
payment_terms['Consumption Date'] = np.where(payment_terms['PDD influence on WC consumption'] >= 0,
                                                payment_terms['Consumption Date'] +
                                                pd.to_timedelta(abs(payment_terms['Total WC Consumption period']) -1, 'D'),
                                                payment_terms['Consumption Date'] - pd.to_timedelta(1, 'D') +
                                            pd.to_timedelta(payment_terms['TTD influence on WC consumption'], 'D'))

payment_terms.loc[:,'Consumption Date'] -= pd.to_timedelta(payment_terms.groupby(level=0).cumcount(),'D')
payment_terms = payment_terms.reset_index(drop=True)
payment_terms['Consumption Type'] = 'Payment Terms'

# filter date
payment_terms = payment_terms[(payment_terms['Consumption Date'] > '2016-12-31')]
payment_terms = payment_terms.reset_index(drop=True)

payment_terms.shape

(21396, 57)

# Tableau data

In [17]:
df_output = pd.concat([stock_in_transit, payment_terms])
df_output = df_output.reset_index(drop=True)
df_output.shape

(52130, 57)

# Selection for others

In [18]:
df_agg = df_output.groupby(['Consumption Date', 'Strategy']).sum().reset_index()
df_agg = df_agg[['Consumption Date', 'Strategy', 'WC Amount']]
df_agg['WC Amount'] = df_agg['WC Amount']*-1

#filter the date
df_agg = df_agg[(df_agg['Consumption Date'] <= radar_date)]


# Import Actuals

In [19]:
#df_actuals = pd.read_excel('0FLATANALYTICALexportETRM.XLSX')

In [20]:
# send the columns to list
#cols_list =df_actuals.columns.tolist()

In [21]:
#transposing data
#df_test = pd.melt(df_actuals,id_vars = cols_list[:6], value_vars=cols_list[6:])

In [22]:
crude_act = pd.read_excel('1ALLBOOKWCActuals - ETRM.xlsm', sheetname='CRUDE', header=None,
                          names = ["Company Code", "Consumption Date", "Strategy", "WC Amount"])

#selecting cols
crude_act = crude_act[['Consumption Date', 'Strategy', 'WC Amount']]
crude_act['WC Amount'] = crude_act['WC Amount']/1000000

# Joining Analytical and Actuals --> Other

In [23]:
# joining the two dataframes

df_others = pd.concat([df_agg, crude_act])
df_others.shape

(426715, 3)

In [24]:
# groupby and exclude the small amounts

df_others = df_others.groupby(['Consumption Date', 'Strategy']).sum().reset_index()
df_others = df_others[(df_others['WC Amount'] > 0.1) | (df_others['WC Amount'] < -0.1) ]
df_others = df_others.reset_index(drop=True)
df_others.shape

(64519, 3)

In [25]:
# only the radar date and defining days_count

df_others_radar_date = df_others[(df_others['Consumption Date'] == radar_date)]
df_others_radar_date = df_others_radar_date.reset_index(drop=True)
days_count = df_others_radar_date.shape[0]

In [26]:
print(days_count)
print(radar_date)
print(add_days)
print(end_date)

267
2017-09-12
49
2017-10-31


In [27]:
# replicating the data until the last day of radar report

df_others_replication = pd.concat([df_others_radar_date]*add_days)
df_others_replication = df_others_replication.reset_index(drop=True)
df_others_replication.shape

(13083, 3)

In [28]:
# changing the dates: from radar date + 1 day until last day of next month

for x in xrange(0,add_days):
    df_others_replication.iloc[x*days_count:days_count*(x+1)]['Consumption Date'] = df_others_replication.iloc[x*days_count:days_count*(x+1)]['Consumption Date'] + pd.to_timedelta(x+1,'D')

In [29]:
df_others_replication['WC Amount'].sum()

1329.1927175100036

In [30]:
df_others = pd.concat([df_others, df_others_replication])

# Export

In [31]:
# before macros
#writer = pd.ExcelWriter('before macros.xlsx')
#df.to_excel(writer, 'data')
#writer.save()

In [32]:
# stock in transit
#writer = pd.ExcelWriter('stock_in_transit.xlsx')
#stock_in_transit.to_excel(writer, 'data')
#writer.save()

In [33]:
# payment terms
#writer = pd.ExcelWriter('payment_terms.xlsx')
#payment_terms.to_excel(writer, 'data')
#writer.save()

In [34]:
# tableau data
#writer = pd.ExcelWriter('tableau data.xlsx')
#df_output.to_excel(writer, 'Tableau')
#writer.save()

In [35]:
# others
#writer = pd.ExcelWriter('others.xlsx')
#df_others.to_excel(writer, 'data')
#writer.save()

# Strategy database

In [59]:
df_strategy = df[['Strategy', 'Strategy name', 'Global Book name', 'Line of Business', 'Book Name', 'Portfolio name']].groupby('Strategy')
df_strategy.drop_duplicates()
#df_strategy.drop_duplicates()
df_strategy.head(5)

AttributeError: Cannot access callable attribute 'drop_duplicates' of 'DataFrameGroupBy' objects, try using the 'apply' method

In [49]:
df.dtypes

Consumption Date                   datetime64[ns]
Consumption Type                           object
Total WC Consumption period                 int64
TTD influence on WC consumption             int64
PDD influence on WC consumption             int64
TTD difference                              int64
PDD difference                              int64
Sale Payment Terms                          int64
Buy Payment Terms                           int64
Consumption reason                         object
WC Amount                                 float64
Buy/Sell (-)                               object
Buy/Sell (+)                               object
Due date (-)                               object
Due date (+)                               object
Title transfer date (-)                    object
Title transfer date (+)                    object
Strategy                                    int64
Strategy name                              object
Company Name                               object
